In [1]:
import pandas as pd
import numpy as np
import cdflib
from sgp4.api import Satrec, jday
from datetime import datetime
import xarray as xr
import re
import os

whi_int = pd.read_pickle('whi_int.pkl')
all_conj = pd.read_pickle('all_conj.pkl')

In [2]:
whi_int

,dt1,dt2
120,2011-07-23 01:40:00+00:00,2011-07-23 01:52:00+00:00
122,2011-07-27 14:22:00+00:00,2011-07-27 14:34:00+00:00
123,2011-07-29 20:47:00+00:00,2011-07-29 20:58:00+00:00
124,2011-08-01 03:13:00+00:00,2011-08-01 03:23:00+00:00
125,2011-08-03 09:39:00+00:00,2011-08-03 09:48:00+00:00
127,2011-08-07 22:28:00+00:00,2011-08-07 22:38:00+00:00
128,2011-08-10 04:52:00+00:00,2011-08-10 05:02:00+00:00
129,2011-08-12 11:16:00+00:00,2011-08-12 11:26:00+00:00
130,2011-08-14 17:42:00+00:00,2011-08-14 17:50:00+00:00
131,2011-08-17 00:07:00+00:00,2011-08-17 00:15:00+00:00


#### Outputs conjunctions where WHISPER data is available, within the range set by 'max_dist'. Essentially searches for common datetimes between the conjunctions list and the WHISPER availability list. 

In [154]:
max_dist = 200

conj = all_conj.loc[(all_conj['conj-dist'] < max_dist)].sort_values(by='conj-time').reset_index(drop=True)
whi_int = whi_int.reset_index(drop=True)
whi_int['dt1'] = whi_int['dt1'].dt.tz_localize(None)
whi_int['dt2'] = whi_int['dt2'].dt.tz_localize(None)

all_index = []
for i in whi_int.index:
    a = np.nonzero(conj['conj-time'] > whi_int['dt1'][i])[0]
    b = np.nonzero(conj['conj-time'] < whi_int['dt2'][i])[0]
    index = list(set(a).intersection(b))
    if len(index) == 0:
        continue
    else:
        all_index = np.concatenate((all_index, index))

# conj.loc[all_index].sort_values(by='conj-time').reset_index(drop=True).to_pickle('whi_conj_200.pkl')

In [4]:
print(len(set(pd.read_pickle('whi_conj_200.pkl')['sat-ID'])))

pd.read_pickle('whi_conj_200.pkl')

43


,sat-ID,int-index,conj-time,conj-dist
0,4578,122.0,2011-07-27 14:24:06,122.685956
1,25039,122.0,2011-07-27 14:24:59,65.421226
2,20432,122.0,2011-07-27 14:25:50,166.397907
3,16262,122.0,2011-07-27 14:27:34,171.232289
4,30777,123.0,2011-07-29 20:51:22,159.361335
5,28809,123.0,2011-07-29 20:54:21,71.838822
6,6392,125.0,2011-08-03 09:47:12,155.521221
7,20442,125.0,2011-08-03 09:47:50,185.874829
8,11060,127.0,2011-08-07 22:28:02,154.663968
9,26548,127.0,2011-08-07 22:29:10,138.173452
